**Importing Some Basic Libraries**

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import os
path1 = r'C:\Users\hp\Desktop\Capstone project\dataset'
print(os.listdir(path1))

import sys
pathcv = r'C:\Users\hp\Anaconda3\envs\opencv-env\Lib\site-packages'
sys.path.append(pathcv)

import cv2
from sklearn import utils

from skimage.transform import rescale, resize
import tensorflow as tf

from keras import backend as K
from keras.preprocessing.image import load_img
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

**Defining Image Path and Assigning Labels.**

In [ ]:
def image_path(img_type, img_number):
    
    #image_type: 'images' #original image, 'annotations' #crop-weed label, 'mask' #vegetation segmentation
    #image_number: the number on the image name
    
    image_name = img_type[:-1]
    if img_number < 10:
        path = 'C:/Users/hp/Desktop/Capstone project/dataset/'+str(img_type)+'/00'+str(img_number)+'_'+str(image_name)+'.png'
    else:
        path = 'C:/Users/hp/Desktop/Capstone project/dataset/'+str(img_type)+'/0'+str(img_number)+'_'+str(image_name)+'.png'
    return path

# generating labels for each image
def label_generator(number):
    annotation = cv2.imread(image_path('annotations', number))
    height = annotation.shape[0]
    width = annotation.shape[1]
   # channel = annotation.shape[2]
    labels = np.zeros((height, width, 3))
    for i in range(height):
        for j in range(width):
            if np.all(annotation[i,j,:] == np.array([0,255,0])):
                labels[i,j,0] = 1
            elif np.all(annotation[i,j,:] == np.array([0,0,255])):
                labels[i,j,1] = 1
            elif np.all(annotation[i,j,:] == np.array([0,0,0])):
                labels[i,j,2] = 1
    return labels

In [ ]:
# rescaling the image

# we rescale the image so as to get a smaller image, which reduces our calculations and in longer run also reduces 
# time complexity and computations.

image = cv2.imread(image_path('annotations',1))
image_rescaled = rescale(image, 1.0 / 6.0, anti_aliasing=True)
plt.imshow(image_rescaled)

In [ ]:
# Load two images
img1 = cv2.imread(image_path('images',1))
img2 = cv2.imread(image_path('masks',1))

# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols ]

# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2,img2,mask = mask)

# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst
plt.imshow(img1)

In [ ]:
label = label_generator(1)
plt.figure()

#plt.imshow(label)
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(label[:,:,i])

In [ ]:
x_train = np.zeros((120, 161, 216, 3))
y_train = np.zeros((120, 161, 216, 3))
x_test = np.zeros((20, 161, 216, 3))
y_test = np.zeros((20, 161, 216, 3))

plt.figure(figsize=(8,5))

for i in range(40):
    image = cv2.imread(image_path('images',i+1))
    image_rescaled = rescale(image, 1.0 / 6.0, anti_aliasing=True)
    label = label_generator(i+1)
    label_rescaled = rescale(label, 1.0 / 6.0, anti_aliasing=True)
    x_train[i,:,:,:] = image_rescaled
    y_train[i,:,:,:] = label_rescaled
    x_train[40+i,:,:,:] = np.fliplr(image_rescaled)
    y_train[40+i,:,:,:] = np.fliplr(label_rescaled)
    x_train[80+1,:,:,:] = np.flipud(image_rescaled)
    y_train[80+i,:,:,:] = np.flipud(label_rescaled)
    plt.subplot(8,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i,:,:,:])
    
for i in range(20):
    image = cv2.imread(image_path('images',i+41))
    image_rescaled = rescale(image, 1.0 / 6.0, anti_aliasing=True)
    label = label_generator(i+41)
    label_rescaled = rescale(label, 1.0 / 6.0, anti_aliasing=True)
    x_test[i,:,:,:] = image_rescaled
    y_test[i,:,:,:] = label_rescaled
    

In [ ]:
plt.figure()
#plt.imshow(y_train[i,:,:])
for i in range(40):
    plt.subplot(8,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(y_train[i,:,:,2])

In [ ]:
# functions for metrics evaluation

# defining a softmax function for our model
def depth_softmax(matrix):
    sigmoid = lambda x: 1 / (1 + K.exp(-x))
    sigmoided_matrix = sigmoid(matrix)
    softmax_matrix = sigmoided_matrix / K.sum(sigmoided_matrix, axis=0)
    return softmax_matrix

# defining a recall function for our model
# recall means the accuracy of predicting the negative class.
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# defining a precision function for our model
# precision means the accuracy of predicting the positive class
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# defining a f1 function for computing the f1 score
# f1 score  = 2* precision*recall/precision + recall
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

'''
Mean Intersection-Over-Union is a common evaluation metric for semantic image segmentation, 
which first computes the IOU for each semantic class and then computes the average over classes.
IOU is defined as follows: IOU = true_positive / (true_positive + false_positive + false_negative).
'''
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

'''
Binary cross entropy is just a special case of categorical cross entropy. The equation for binary cross entropy
loss is the exact equation for categorical cross entropy loss with one output node. For example, binary cross 
entropy with one output node is the equivalent of categorical cross entropy with two output nodes.
'''
def weighted_binary_cross_entropy(y_true, y_pred):
    w = tf.reduce_sum(y_true)/tf.cast(tf.size(y_true), tf.float32)
    real_th = 0.5-(1.0/2.0)
    tf_th = tf.fill(tf.shape(y_pred), real_th) 
    tf_zeros = tf.fill(tf.shape(y_pred), 0.)
    return (1.0 - w) * y_true * - tf.log(tf.maximum(tf.zeros, tf.sigmoid(y_pred) + tf_th)) + (1- y_true) * w * -tf.log(1 - tf.maximum(tf_zeros, tf.sigmoid(y_pred) + tf_th))
#return weighted_binary_cross_entropy

# assigning weights to target according to output
def class_weighted_pixelwise_crossentropy(target, output):
    output = tf.clip_by_value(output, 10e-8, 1.-10e-8)
    weights = [0.8, 0.2]
    return -tf.reduce_sum(target * weights * tf.log(output))


In [ ]:

# defining an activation function
def BatchActivate(x):
    # using batch normalization
    x = BatchNormalization()(x)
    # using relu activation function
    x = Activation('relu')(x)
    return x

# defining a convolutional block
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    # using a strides of 1 by 1 with a activation function
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

# defining a residual block
def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [ ]:
# defining our model

def unet_res(n_classes = 1, start_neurons = 16, DropoutRatio = 0.5, img_height=161,img_width=216):
    # 101 -> 50
    # defining the input layer
    input_layer = Input((img_height, img_width, 3))
    # padding, initially adding zeros to input layer
    zero_pad = ZeroPadding2D(padding=((7,8),(4,4)))(input_layer)
    
    # starting with convolutional layer
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(zero_pad)
    # adding residual blocks
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1, True)
    
    # max pooling applied 
    pool1 = MaxPooling2D((2, 2))(conv1)
    # dropout to avoid overfitting
    pool1 = Dropout(DropoutRatio/2)(pool1)

    # 50 -> 25
    # starting with convolutional layer
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    # adding residual blocks
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2, True)
    # adding pooling
    pool2 = MaxPooling2D((2, 2))(conv2)
    # dropout to avoid overfitting
    pool2 = Dropout(DropoutRatio)(pool2)

    # 25 -> 12
    # starting with convolutional layer
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    # adding residual blocks
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4, True)
    # adding max pooling
    pool3 = MaxPooling2D((2, 2))(conv3)
    # adding dropout
    pool3 = Dropout(DropoutRatio)(pool3)

    # 12 -> 6
    # starting wit convolutional layer
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    # adding residual blocks
    conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8, True)
    # adding max pooling layer
    pool4 = MaxPooling2D((2, 2))(conv4)
    # adding dropout
    pool4 = Dropout(DropoutRatio)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 16)
    convm = residual_block(convm,start_neurons * 16, True)
    
    # 6 -> 12
    # transpose convolutional layer 4
    # going backward of convolution operation, it is the core idea of transposed convolution.
    # used as up sampling method
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    # concatinating the convolutional 4 and transposed convolutional layers 4
    uconv4 = concatenate([deconv4, conv4])
    # adding dropout
    uconv4 = Dropout(DropoutRatio)(uconv4)
    
    # adding it to the architecture 
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8, True)
    
    # 12 -> 25
    # transposed convolutional layer 3
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv4)
    # adding convolutional layer 3 to transposed convolutional layer 3
    uconv3 = concatenate([deconv3, conv3])
    # adding dropout
    uconv3 = Dropout(DropoutRatio)(uconv3)
    
    # adding it to the architecture
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4, True)

    # 25 -> 50
    # transposed convolutional layer 2
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    # adding convolutional layer 2 and transposed convolutional layer 2
    uconv2 = concatenate([deconv2, conv2])
    # adding dropout
    uconv2 = Dropout(DropoutRatio)(uconv2)
    
    # adding it to the architecture
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2, True)
    
    # 50 -> 101
    # transposed convolutional layer 1
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    # adding convolutional layer 1 and transposed convolutional layer 1
    uconv1 = concatenate([deconv1, conv1])
    # adding dropout
    uconv1 = Dropout(DropoutRatio)(uconv1)
    
    # adding it to the architecture
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1, True)
    
    #uconv1 = Dropout(DropoutRatio/2)(uconv1)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    # output layer
    output_layer_noActi = Conv2D(n_classes, (1,1), padding="same", activation=None)(uconv1)
    output_layer = Cropping2D(cropping=((7,8),(4,4)))(output_layer_noActi)
    
    # defining the output layer
    # reshaping the images to defined width and height
    if n_classes == 1:
        output_layer = (Reshape((img_height, img_width),
                          input_shape=(img_height, img_width, 3)))(output_layer)
    else:
        output_layer = (Reshape((img_height, img_width, n_classes),
                          input_shape=(img_height, img_width, 3)))(output_layer)
    
    # defining the activation for output layer to be sigmoid
    output_layer =  Activation('sigmoid')(output_layer)
    
    # finalizing our model
    model = Model(input_layer, output_layer)
    
    return model

In [ ]:
# looking at the summary of the training model

trial_model = unet_res(n_classes = 1, start_neurons=16)
trial_model.summary()

In [ ]:

# getting the bg model
bg_model = unet_res(n_classes=1)

# compiling the model
bg_model.compile(loss = 'binary_crossentropy',
             optimizer = 'Adam',
             metrics = [f1, mean_iou])

# using early stoppinng and reduce learning on plateau to avoid overfitting
callbacks_bg = [
    EarlyStopping(patience=5, verbose=1),
    ReduceLROnPlateau(patience=3, verbose=1),
    ModelCheckpoint('model-bg-cwfid.h5', verbose=1, save_best_only=True)
]

# feeding training data to bg model
model_bg_history = bg_model.fit(x = x_train,
                         y = y_train[:,:,:,2],
                         batch_size = 5,
                         epochs = 20,
                         verbose = 1,
                         validation_split = 0.3,
                          callbacks = callbacks_bg
                         )


In [ ]:

# getting the weed model
weed_model = unet_res(n_classes=1)

# compiling the compile
# optimizer used is adam optimizer
# metrics used are f1 and mean_iou as defined above
weed_model.compile(loss = 'binary_crossentropy',
             optimizer = 'Adam',
             metrics = [f1, mean_iou])

# using early stopping to stop when the model stops learning
callbacks_weed = [
    EarlyStopping(patience=5, verbose=1),
    ReduceLROnPlateau(patience=3, verbose=1),
    ModelCheckpoint('model-weed-cwfid.h5', verbose=1, save_best_only=True)
]

# feeding the model with training data
model_weed_history = weed_model.fit(x = x_train,
                         y = y_train[:,:,:,1],
                         batch_size = 5,
                         epochs = 20,
                         verbose = 1,
                         validation_split = 0.2,
                          callbacks = callbacks_weed
                         )

In [ ]:

# getting the crop model
crop_model = unet_res(n_classes=1)

# compiling the crop_model with adam optimizer
crop_model.compile(loss = 'binary_crossentropy',
             optimizer = 'Adam',
             metrics = [f1, mean_iou])

# using early stopping to stop when the model stops learning
callbacks_crop = [
    EarlyStopping(patience=5, verbose=1),
    ReduceLROnPlateau(patience=3, verbose=1),
    ModelCheckpoint('model-crop-cwfid.h5', verbose=1, save_best_only=True)
]

# feeding training data to crop_model
model_crop_history = crop_model.fit(x = x_train,
                         y = y_train[:,:,:,0],
                         batch_size = 5,
                         epochs = 20,
                         verbose = 1,
                         validation_split = 0.3,
                          callbacks = callbacks_crop
                         )

In [ ]:
# predictions for all the three models

pred_bg = bg_model.predict(x_test)
pred_weed = weed_model.predict(x_test)
pred_crop = crop_model.predict(x_test)

In [ ]:
# looking at the images predicted by all model for weeds and crops.

i = 15
plt.figure()
plt.subplot(2,3,1)
plt.imshow(pred_bg[i,:,:])
plt.subplot(2,3,4)
plt.imshow(y_test[i,:,:,2])
plt.subplot(2,3,2)
plt.imshow(pred_weed[i,:,:])
plt.subplot(2,3,5)
plt.imshow(y_test[i,:,:,1])
plt.subplot(2,3,3)
plt.imshow(pred_crop[i,:,:])
plt.subplot(2,3,6)
plt.imshow(y_test[i,:,:,0])


In [ ]:

# combining all the model
full_model = unet_res(n_classes=3)

# compiling the model
full_model.compile(loss = 'binary_crossentropy',
             optimizer = 'Adam',
             metrics = [f1, mean_iou])

# using early stopping, reduce-learning rate on plateau to avoid over fititng
callbacks_full = [
    EarlyStopping(patience=8, verbose=1),
    ReduceLROnPlateau(patience=5, verbose=1),
    ModelCheckpoint('model-full-cwfid.h5', verbose=1, save_best_only=True)
]


# feeding training data to the model 
model_full_history = full_model.fit(x = x_train,
                         y = y_train,
                         batch_size = 5,
                         epochs = 30,
                         verbose = 1,
                         validation_split = 0.2,
                          callbacks = callbacks_full
                         )

In [ ]:
# predicting for the full combined model
pred_full = full_model.predict(x_test)

In [ ]:
# looking at the results predicted by our model

i = 15
plt.figure()
plt.subplot(2,3,1)
plt.imshow(pred_full[i,:,:,2])
plt.subplot(2,3,4)
plt.imshow(y_test[i,:,:,2])
plt.subplot(2,3,2)
plt.imshow(pred_full[i,:,:,1])
plt.subplot(2,3,5)
plt.imshow(y_test[i,:,:,1])
plt.subplot(2,3,3)
plt.imshow(pred_full[i,:,:,0])
plt.subplot(2,3,6)
plt.imshow(y_test[i,:,:,0])